In [1]:
import numpy as np
from scipy.signal import find_peaks
import pandas as pd
from snr_functions import *

In [2]:
def load_results(gal_type,dataset,runtype):
    # load results
    path = f"/Users/aizaali/Desktop/desi_data/coadd_results/{gal_type}_{dataset}_{runtype}_coadd_results.npz"
    data = np.load(path)

    wave = data['wavelength']    
    valid_bins = data['valid_bins']
    # smoothed residuals (gaussian)
    snr_narrow = data['snr_narrow']  #3 A
    # snr_broad = data['snr_broad']  #15 A
    noise = data['noise_narrow']

    noise = noise[valid_bins]
    valid_wave = wave[valid_bins]
    valid_snr_narrow = snr_narrow[valid_bins]
    result = set()
    peaks, _ = find_peaks(valid_snr_narrow)
    valleys, _ = find_peaks(-valid_snr_narrow)
    extrema = np.sort(np.concatenate((peaks, valleys)))
    for i in extrema:
        current_snr = valid_snr_narrow[i]
        current_wave = valid_wave[i] 
        if abs(current_snr) >= 2.5:
            to_add = (current_snr,current_wave)
            result.add(to_add)

    print('done loading results')
    return result

In [ ]:
def main():
    all_types = ['BGS_loa_galaxy', 'LRG_loa_galaxy', 'BGS_loa_skyfiber', 'LRG_loa_skyfiber',
                'BGS_iron_galaxy', 'LRG_iron_galaxy', 'BGS_iron_skyfiber', 'LRG_iron_skyfiber']
    # initialize results dictionary
    results = {}
    for type in all_types:
        input_list = type.split('_')
        gal_type = input_list[0]
        dataset = input_list[1]
        runtype = input_list[2]
        key = f"{gal_type}_{dataset}_{runtype}_result"
        results[key] = load_results(input_list[0],input_list[1],input_list[2])
        print(f"{gal_type}_{dataset}_{runtype} has {len(results[key])} SNR points with value >= 2.5")
    
    print("crossmatching....")
    matches = crosscheck(results)

    print(f"Found {len(matches)} lines recovered in ≥ 2 datasets")
    print("Ignoring known lines...")
    final_result = reject_known_lines(matches,1)

    df_filtered = pd.DataFrame(final_result)

    df_filtered.to_csv('test_crossmatch_results.csv', index=False)
    print("Results saved to 'test_crossmatch_results.csv'")

    for m in final_result:
        print(
            f"λ = {m['mean_wave']:.2f} Å | "
            f"N = {m['n_recovered']} | "
            f"datasets = {m['datasets']}"
        )

In [4]:
main()

done loading results
BGS_loa_galaxy has 47 SNR points with value >= 2.5
done loading results
LRG_loa_galaxy has 40 SNR points with value >= 2.5
done loading results
BGS_loa_skyfiber has 29 SNR points with value >= 2.5
done loading results
LRG_loa_skyfiber has 26 SNR points with value >= 2.5
done loading results
BGS_iron_galaxy has 45 SNR points with value >= 2.5
done loading results
LRG_iron_galaxy has 40 SNR points with value >= 2.5
done loading results
BGS_iron_skyfiber has 30 SNR points with value >= 2.5
done loading results
LRG_iron_skyfiber has 27 SNR points with value >= 2.5
crossmatching....
Found 57 lines recovered in ≥ 2 datasets
Ignoring known lines...
Results saved to 'test_crossmatch_results.csv'
λ = 3934.80 Å | N = 5 | datasets = ['BGS_iron_galaxy_result', 'BGS_loa_galaxy_result', 'BGS_loa_skyfiber_result', 'LRG_iron_galaxy_result', 'LRG_loa_galaxy_result']
λ = 6586.00 Å | N = 4 | datasets = ['BGS_iron_galaxy_result', 'BGS_loa_galaxy_result', 'LRG_iron_galaxy_result', 'LRG